Мы узнали про достаточное количество алгоритмов машинного обучения и про то, как оценивать качество алгоритмов

Следующий шаг - научиться настраивать алгоритмы для получения максимального качества


# Feature engeneering

В задачи unsupervised и supervised объединяет общий элемент - матрица *объекты* $\times$ *признаки* размерность $m \times n$, где $m$ - число объектов, а $n$ - число признаков

$$
X = \left[
\begin{array}{cccc}
x_{11} & x_{12} & \ldots & x_{14} \\
x_{21} & x_{22} & \ldots & x_{14} \\
\ldots & \ldots & \ldots & \ldots \\
x_{m1} & x_{m2} & \ldots & x_{mn} \\
\end{array}
\right]
$$

Таким образом, каждый объект описан признаками(фичами) в количестве $n$ штук: $x^i = (x_1, x_2, \ldots, , x_n)$. Мы уже знаем, что фичи бывают численными и категориальными.

Фичи можно назвать "топливом" алгоритмов машинного обучения. Хорошие фичи позволят повысить качество решения задачи, примерно как на картинке

![ml_blackbox](img/ml_blackbox.png)


В алгоритмах машинного обучения и анализа данных часто встречаются требования к фичам входных данных
* распределение данных
* масштаб

Перед аналитиком часто стоит задача трансформации (преобразования) входных данных таким образом, чтобы удовлетворить условиям алгоритма. Игнорирование требований к входным данным приводит некорректным выводам, это омновной принцип ML (и не только ML): **garbage in - garbage out**. Процесс "придумывания" фичей называется feature engineering

В этом занятии поговорим о том, как трансформировать исходный csv файл в набор фичей

## Непрерывные переменные

В задачах линейной регрессии такие трансформации особенно важны - чтобы линейная регрессия хорошо работала должны выполняться следующие требования:

* остатки регрессии должны иметь нормальное (гауссово) распределение
* все фичи должны быть примерно в одном масшабе

При подготовке данных для обучения линейной регрессии применяются следющие приемы: масштабирование и нормализация.

### Монотонные преобразования

Существуют чисто инженерные приёмы первичной обработки данных, например для борьбы с большими по модулю значениями обычно используют

* логарифмирование np.log
* извлечение квадратного корня np.sqrt

Оба этих преобразования являются *монотонными*, т.е. они меняют абсолютные значения, но сохраняют порядок величин.

### z-score

Более интересный метод - это Standart Scaling или Z-score normalization. Это преобразование позволяет "сгладить" данные, избавить их от выбросов. Для этого есть инструмента [есть реализация в sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.stats import shapiro

data = pd.read_csv('data/task.csv')
print(data.head(3))

raw_data = data[data.columns.values[-1]].values.astype(np.float32)
print("Сырой датасет: %s" % raw_data)
print("stat = %s, p-value=%s\n" % shapiro(raw_data) )

transformed_data = StandardScaler().fit_transform(raw_data.reshape(-1, 1)).reshape(-1)
print("z-transform датасет: %s" % transformed_data)
print("stat = %s, p-value=%s\n" % shapiro(transformed_data) )

print("Проверка на нормальность p_1 > p_2: %s" % (shapiro(transformed_data)[1] > shapiro(raw_data)[1]))

         Код                                               Тема  \
0  HYDRA-535  Пробрасывать пользовательское распределение pa...   
1  HYDRA-534   Гибридный рекомендатель с multi-channel feedback   
2  HYDRA-532         Джоба в дженкинсе для расчёта динамики РВП   

       Компонент  Затрачено в часах  
0        echidna                  1  
1          hydra                  3  
2  hydramatrices                  2  
Сырой датасет: [ 1.  3.  2.  4.  2. 10.  2.  5.  2.  2.  1.  7.  5.  2.  5. 16. 10.  3.
 24.]
stat = 0.7262855768203735, p-value=0.00011633868416538462

z-transform датасет: [-0.79860216 -0.4497874  -0.6241948  -0.27538007 -0.6241948   0.7710641
 -0.6241948  -0.10097268 -0.6241948  -0.6241948  -0.79860216  0.24784204
 -0.10097268 -0.6241948  -0.10097268  1.8175083   0.7710641  -0.4497874
  3.2127674 ]
stat = 0.7262856960296631, p-value=0.00011633900430751964

Проверка на нормальность p_1 > p_2: True


Тест Шапиро-Уилка показывает, что гипотеза о нормальном распределении данных стала чуть более вероятной, чем до "Z-score" нормализации.

### min-max normalization

Другой распространённый метод называется MinMax Scaling. Этот метод переносит все точки на отрезок [0-1]
$$
X_{scaled} = \frac{X - X_{\min}}{X_{\max} - X_{\min}}
$$

In [3]:
from sklearn.preprocessing import MinMaxScaler


print("Сырой датасет: %s" % raw_data)

transformed_data = MinMaxScaler().fit_transform(raw_data.reshape(-1, 1)).reshape(-1)

print("Min-Max scale датасет: %s" % transformed_data)

Сырой датасет: [ 1.  3.  2.  4.  2. 10.  2.  5.  2.  2.  1.  7.  5.  2.  5. 16. 10.  3.
 24.]
Min-Max scale датасет: [0.         0.08695652 0.04347826 0.13043478 0.04347826 0.39130437
 0.04347826 0.17391305 0.04347826 0.04347826 0.         0.2608696
 0.17391305 0.04347826 0.17391305 0.65217394 0.39130437 0.08695652
 1.        ]



## Категориальные переменные

Категориальная переменная - это набор меток (классов). В приложенном затасете по задам столбец `Компонент` - категориальная фича, а `Затрачено в часах` - непрерывная


In [30]:
import pandas as pd

df = pd.read_csv('data/task.csv')

df.head()

,Код,Тема,Компонент,Затрачено в часах
0,HYDRA-535,Пробрасывать пользовательское распределение pa...,echidna,1
1,HYDRA-534,Гибридный рекомендатель с multi-channel feedback,hydra,3
2,HYDRA-532,Джоба в дженкинсе для расчёта динамики РВП,hydramatrices,2
3,HYDRA-531,Интеграция Hydra с Gamora,hydramatrices,4
4,HYDRA-530,Тестируем интеграцию с Jira,hydra,2


Посчитаем количество различных меток в поле "Компонент"

In [31]:
df['Компонент'].value_counts()

hydra            11
hydramatrices     6
echidna           1
hydra_utils       1
Name: Компонент, dtype: int64

### Кодирование One-Hot

Кодируем вектор, где все нули и одна единица

In [32]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')

one_hot_encoded = ohe.fit_transform(df[['Компонент']])

one_hot_encoded.toarray()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

### Hashing trick

В случае, когда признаков слишком много, применяют хеширование



In [36]:
for label in df['Компонент'].unique():
    #print(label, '->', hash(label) % 8 )
    print(label, '->', hash(label) % 12 )

echidna -> 1
hydra -> 2
hydramatrices -> 9
hydra_utils -> 6


Премер хеширования (с формулами!) в [Лекциях от ВШЭ](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture06-linclass.pdf)

Прекрасный разбор есть на хабре в статье от [ODS про feature engineering](https://habr.com/ru/company/ods/blog/326418/#rabota-s-kategorialnymi-priznakami-label-encoding-one-hot-encoding-hashing-trick)

## Тексты


Bpdktrкак обработка естественного языка (англ. Natural Language Processing, NLP). NLP изучает проблемы компьютерного анализа естественных языков - т.е. языков, которые для общения используют люди (а не придуманных искусственно (например, азбука Морзе - язык, придуманный искусственно). Подробнее о том, зачем нужен NLP и где именно возникает задача обработки естественного языка мы поговорим в Уроке 1.

Тексты - один из самых доступных и объёмных источников данных. Современный человек потребляет огромное количество контента: фото, звук и текст. Через визуальный канал, т.е. с помощью  фото  и видео, мы получаем 80% всей информации, поэтому такой большой объём курса уделён компьютерному зрению. На втором месте по объёму передаваемой информации - текстовые данные.

Например, если у вас интернет-магазин, то для анализа доступны

* текстовые описания товаров
* пользовательские комментарии
* диалоги с продавцом-консультантом в чатике

Текстовую информацию просто хранить, поэтому проекты накапливают огромные наборы данных такого рода и очень хотят извлекать из этих объёмов полезную информацию.

Как специалист по ML в начале карьеры вы, скорее всего, встретите ряд “классических” задач - например, определение тональности (настроения) текста или классификации сообщений spam/not spam - для таких задач используются подходы, основанные на подсчёте статистик по встречающимся в тексте словам.
Однако, есть и другие, более сложные задачи. 

Для решения применяются различные архитектуры нейросетей (RNN, LSTM) - это мощные инструменты, которые позволяют решать сложные задачи, например: 

* извлечения именованных сущностей ([NER](https://habr.com/ru/post/414175/), Named-Entity Recognizing)
* автоматизированного перевода (например, сервис *google translate* производит перевод с помощью глубоких сетей)
* Speech Recognition - распознавание речи, трансляция из аудио в текстовый вид
* Natural Language Generation - генерация текстов, например можно генерировать подписи к картинкам

У обработки естественного языка есть ряд особенностей:

* необходимо размечать большой объём данных для обучения с учителем. Допустим, хотим отделять спам-сообщения от не спама. Вам нужно найти людей, которые прочитают все смс, которые удалось собрать и отметят те из них, которые являются спамом - текстов обычно очень много и разметка данных может оказаться дорогим удовольствием
* модель, обученную на одном языке невозможно использовать для другого языка
* важен как синтаксис, так и семантика (смысл). Например, во фразе: «Вот списки студентов, которые сдали зачет по физике» определение «которые сдали зачет по физике» относится к студентам, а в предложении: «Вот списки студентов, которые лежали в шкафу у декана»  структура фразы (тот самый синтаксис) такая же, как и предыдущей - но определение уже относится не к студентам, а к листкам бумаги. От компьютера мы хотим добиться, чтобы смыл обеих фраз был “понят” одинаково хорошо.

Кроме того, для текстов на естественном языке довольно сложно проводить предобработку данных, этот этап сильно зависит от задачи, которую вы  решаете. Так, например, для задачи анализа тональности текста знаки препинания, скорее всего, не важны. Однако, для задачи извлечения именованных сущностей (именованная сущность - это имя собственное - например название организации или географического объекта) удалять знаки препинания не рекомендуется - это может привести к потере важной информации. Например если из фразы `Мы пошли обедать в “Берёзку”` если удалить все знаки препинания (кавычки) и заглавную букву в названии заведения то станет сложнее понять, что речь идёт о кафе.

Обработка текста складывается из двух этапов

* предварительная обработка текста
* векторизация текста

#### Предварительная обработка текста

Перед тем, как обучать модель, данные следует специальным образом подготовить. Подготовка данных включает в себя несколько обязательных этапов

* удалить все нерелевантные символы (например, любые символы, не относящиеся к цифро-буквенным).
* токенизировать текст, разделив его на индивидуальные слова (токены)
* удалить нерелевантные слова — например, упоминания в Twitter или URL-ы.
* перевести все символы в нижний регистр для того, чтобы слова «hello», «Hello» и «HELLO» были схлопнуты в один токен
* исправление ошибок ("молоко" и "молако" - одно слово, но разные токены, не надо так)  
* лемматизация - перевод слова в нормальную (словарную) форму (например, «машина» вместо «машиной»). Существительные должны быть приведены к единственному числу именительного падежа, глаголы - инфинитив и т.д.
* стемминг - процедура, когда от слова переходим к его корню ("помыть" и "мытый" - корень "мыт"). То есть все "помытые" заменяем на "мыт".

Все эти приёмы нужно применять с осторожностью и внимательно следить за тем, как тот или иной приём, применённый к исходному тексту, влияет на качество решения задачи (например, выявлению спама)

Для демонстрации всех этих приёмов загрузим корпус (набор текстов) с твитами о продуктах. Для каждого твита размечена эмоциональная окраска - позитивная, нейтральная или негативная. Примечание: для  обработки текста воспользуемся библиотекой nltk, которая [доступна в anaconda](https://anaconda.org/anaconda/nltk)

In [3]:
import nltk
import string
import pandas as pd

# дополнительный словарь со знаками пунктуации
nltk.download('punkt', download_dir='.')

df = pd.read_csv('data/brand_tweets.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df.drop(df[df.tweet_text.isnull()].index, inplace=True)
print(df.shape)

df.head()

(3904, 3)


[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


Для начала разделим текст на отдельные слова с помощью библиотечной функции word_tokenize, на примере первого документа в корпусе

In [4]:
sample_str = df.tweet_text.values[0]

print('== Исходный текст== \n%s\n\n' % sample_str)

tokenized_str = nltk.word_tokenize(sample_str)
print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.


== Токенизированный текст==
['.', '@', 'wesley83', 'I', 'have', 'a', '3G', 'iPhone', '.', 'After', '3', 'hrs', 'tweeting', 'at', '#', 'RISE_Austin', ',', 'it', 'was', 'dead', '!', 'I', 'need', 'to', 'upgrade', '.', 'Plugin', 'stations', 'at', '#', 'SXSW', '.']


Отфильтруем знаки пунктуации, токены приведём к нижнему регистру

In [5]:
tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
print(tokens)

['wesley83', 'i', 'have', 'a', '3g', 'iphone', 'after', '3', 'hrs', 'tweeting', 'at', 'rise_austin', 'it', 'was', 'dead', 'i', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']


Удаляем стоп-слова, список которых для русского языка можно получить как `stop_words = nltk.corpus.stopwords.words('russian')`. Стоп-слова это "мусорные" слова которые встречаются чрезычайно часто (в каждом предложении) поэтому не несут в себе никакой информации. Такие слова, вобщем-то, нужны только для красивой речи и поэтому можем их смело удалять из текста. Например, этот список стоп-слов я нагуглил в интернете.

In [6]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

filtered_tokens = [i for i in tokens if ( i not in stop_words )]

print(filtered_tokens)

['wesley83', '3g', 'iphone', '3', 'hrs', 'tweeting', 'rise_austin', 'dead', 'need', 'upgrade', 'plugin', 'stations', 'sxsw']


#### Домашнее задание

Реализуем пайплайн в виде функции, при помощи которой обработаем все текстовые описания. Для каждого описания
* проводим токенизацию
* удаляем пунктуацию
* приводим к нижнему регистру
* удаляем стоп-слова


Примените процедуру токенизации к файлу brand_tweets_valid.csv

Сколько уникальных токенов получилось?

In [8]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    
    :param raw_text: исходная текстовая строка
    """
    filtered_tokens = []
    # -- ВАШ КОД ТУТ --
    
    # -----------------
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df.tweet_text.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)

df.tokenized.head()

0    []
1    []
2    []
3    []
4    []
Name: tokenized, dtype: object

Следующий шаг - привести слово в нормальную (словарную) форму. Для русского языка мы уже проводили нормализацию (в занятии из цикла по Аналитике начального уровня) с помощью модуля pyMorphy, который отлично подходит для русского языка

<pre>
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
parsed_token = morph.parse(word)
normal_form = parsed_token[0].normal_form
</pre>

В силу того, что наши твиты на английском языке, то этап нормализации не слишком актуален.



### Векторизация текста: Bag of Words


Мы умеем подготавливать текст к обработке: приводить слова к начальным формам, разделять текст на токены, удалять "мусорные" токены (стоп-слова). Однако, мы знаем, что нейросети работают не с текстом, а с числами. Давайте разбираться, как переводить токены в числа, то есть с тем, как работает векторизация

Bag of Words - это способ перейти от набора токенов к численному вектору. Алгоритм векторизации текста по модели BoW:

1. определяем количество $N$ различных токенов во всех доступных текста - так называемый "словарь"
1. присваиваем каждому токену случайный номер от $0$ до $N$
1. для каждого документа $i$ формируем вектор размерности $N$ - ставим на позицию $j$ количество вхождений токена с номером $j$, которые содержатся в тексте $i$.

Каждый токен мы по сути представляем в виде вектора размерности $N$, который состоит из нулей и всего одной единицы, такое кодирование называется *One-Hot encoding*. А каждый документ это "сумма" всех one-hot векторов входящих в него токенов

Такой подход хорошо иллюстрируется картинкой:

![bow](img/bow.png)

Для каждого элемента получаем вектор из нулей и единиц. При этом размерность словаря обычно составляет несколько десятков тысяч, а количество токенов в одном документе несколько десятков - то есть нулей значительно больше, чем единиц - такие данные называются *разреженными*.

В таком виде данные уже пригодны для работы с нейросетью или любым другим алгоритмом ML, однако есть несколько довольно простых и полезных вещей, которые мы можем сделать и без нейросетей. Давайте сначала разберем их, а потом вернемся к нейросетям. Такое представление текста позволяет решать интересные задачи - например, находить самые похожие друг на друга тексты. Чтобы как-то формализовать понятие "схожести" текстов, вводится понятие *косинусного расстояния* между двумя векторами текстов $a$ и $b$ размерности $N$. С этой метрикой вы [можете познакомиться в Википедии](https://ru.wikipedia.org/wiki/Векторная_модель#Косинусное_сходство ), формула такая для двух векторов $a$ и $b$ с координатами $a_i$ и $b_i$ соответственно:
$$
\text{similarity} = \cos (\theta) = 1 - \frac{\sum_{i=1}^{N}a_ib_i}{\sqrt{\sum_{i=1}^{N}(a_i)^2}\sqrt{\sum_{i=1}^{N}(b_i)^2}}
$$

Интуитивное объяснение для простого случая: два документа полностью совпадают, тогда единички в них стоят на одних и тех же местах - расстояние между ними будет нулевым. Если два текста совершенно не пересекаются, то единички будут стоять на разных местах - расстояние в этом случае равно единице. Самостоятельно реализовывать функцию не нужно - есть готовая реализация в [scipy.spatial.distance.cosine](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.cosine.html)

Векторизуем наш корпус (набор текстов) с помощью класса `CountVectorizer()` (то есть превращаем наборы токенов в наборы векторов)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# инициализируем объект, который токенизирует наш текст
# в качестве единственного аргимента передаём функцию, которую мы написали в Уроке 2
# на разбивает каждый документ на токены
vectorizer = CountVectorizer(tokenizer=tokenize_text)
# применяем наш объект-токенизатор к датафрейму с твитами
document_matrix = vectorizer.fit_transform(df.tweet_text.values)
# результат - матрица, в которой находятся числа, строк в мастрице столько, сколько документов
# а столбцов столько, сколько токенов
document_matrix

ValueError: empty vocabulary; perhaps the documents only contain stop words

Класс `sklearn.feature_extraction.text.CountVectorizer` реализует алгоритм преобразования массива текстовых документов в разреженную матрицу такую, что

* число строк совпадает с количеством документов в исходном датафрейме
* количество столбцов совпадает с количеством различных токенов
* объект `CountVectorizer()` содержит в себе разные вспомогательные элементы - например, словарь соответствия токена и его номера

Полученные вектора можно использовать в алгоритмах второго уровня - например, в задаче классификации отзывов.

Пользуясь матрицей, найдем твит, который максимально похож на первый твит из набора:

In [10]:
source_tweet_index = 0
print(df.iloc[source_tweet_index].tweet_text)

.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.


Вычисляем попарные схожести между элементами разреженной матрицы

In [11]:
from sklearn.metrics import pairwise_distances

tweet_distance = 1-pairwise_distances(document_matrix, metric="cosine")

tweet_distance.shape

NameError: name 'document_matrix' is not defined

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [12]:
import numpy as np

# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[source_tweet_index,:])

sorted_similarity

NameError: name 'tweet_distance' is not defined

Мы получили вектор "схожестей", который содержит индексы похожих твитов, расположенных по убыванию схожести. Больше всего твит похож сам на себя, поэтому возьмём индекс второго по схожести элемента (и далее).

In [13]:
print(df.iloc[0].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[1]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[2]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[3]].tweet_text)

.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.
-------------


NameError: name 'sorted_similarity' is not defined

Мы получили мощный инструмент для анализа текстов - например, мы случайно нашли дубликат твита

Кроме простого подхода, когда мы вычисляем счётчик вхождения токена, можно вычислять более сложную метрику TF-IDF (term frequency - inverse document frequency), которая вычисляется по следующей формуле для токена $t$ и документа $d$:
$$
\text{tf-idf}(t,d) = \text{tf}(t,d)\cdot\text{idf}(t)
$$

Где $\text{tf}(t,d)$ - элемент матрицы, полученной из `CountVectorizer()`, который мы умножаем на величину $\text{idf}(t)$. 

Эта величина показывает количество документов в корпусе  (наборе текстов), в которых был встречен токен $t$:
$$
\text{idf}(t) = \log\frac{1+N}{1+\text{df(t)}} + 1
$$

где $\text{df}(t)$ - количество документов корпуса, в которых был встречен токен $t$. Таким образом мы понижаем веса у слов, которые встречаются почти во всех документах - такие токены являются неинформативными и мусорными, алгоритм понижает их "важность" для анализа.

Алгоритм TF-IDF лучше подходит для анализа текстов и даёт более высокое качество, но более затратен по вычислениям. Как выбрать между этими алгоритмами?

* если токенов менее 10000 используйте TF-IDF
* если токенов более 10000 то *попробуйте* использовать TF-IDF, если не получится - возвращайтесь к CountVectorizer

**Недостатки BoW подхода** Используя алгоритмы вроде Вag of Words, мы теряем порядок слов в тексте, а значит, тексты "i have no cows" и "no, i have cows" будут идентичными после векторизации, хотя и противоположными семантически. Чтобы избежать этой проблемы, можно сделать шаг назад и изменить подход к токенизации: например, использовать N-граммы (комбинации из N последовательных токенов). Обычно по корпусу  (набору текстов) формируются биграммы (последовательности из двух слов) или триграммы (последовательности из трёх слов)

Кроме того, текст можно разбивать не на слова, а на последовательности букв - при таком подходе опечатки будут автоматически учитываться.

#### Домашнее задание

Потренируйтесь в нахождении матрицы схожести для валидационного сета

загрузите brand_tweets_valid.csv
примените объект vectorizer, обученный на датасете brand_tweets.csv (просто скопируйте этот код из урока)
примените функцию pairwise_distances к полученной матрице

In [14]:
import pandas as pd

df_valid = pd.read_csv('data/brand_tweets_valid.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df_valid.drop(df[df.tweet_text.isnull()].index, inplace=True)

# -- ВАШ КОД ТУТ --



* Пользуясь матрицей схожести, полученной на предыдущем этапе, найдите top-5 твитов, похожих на твит валидационного сета с id=14.

У вас есть матрица схожести между объектами. Попробуйте решить задачу поиска дубликатов в тексте

1. Визуализируйте гистограмму значений в матрице схожести
1. Напишите функцию на Python, которая принимает индекс твита, пороговое значение (число от $0.0$ до $1.0$ и матрицу схожести, а затем выводит все твиты, схожесть которых больше, чем пороговое значение